# Notebook to generate `indicators` module

In [11]:
from pathlib import Path

from pprint import pformat

from mintalib import core

from inspect import Signature

from mintalib.samples import sample_prices


class TEXT(str):
    def __repr__(self):
        return self

In [12]:

ROOTDIR = Path.cwd().parent
PKGDIR = ROOTDIR.joinpath("src/mintalib").resolve(strict=True)



In [13]:

prices = sample_prices()


In [14]:
import importlib.util

def new_module(name: str, code: str = None):
    spec = importlib.util.spec_from_loader(name, None)
    module = importlib.util.module_from_spec(spec)
    if code:
        exec(code, module.__dict__)
    return module

In [15]:
def core_functions():
    return list(k for k, v in vars(core).items() if k.isupper() and callable(v))

core_functions()

['AVGPRICE',
 'TYPPRICE',
 'WCLPRICE',
 'MIDPRICE',
 'PRICE',
 'CROSSOVER',
 'CROSSUNDER',
 'FLAG_ABOVE',
 'FLAG_BELOW',
 'FLAG_INVERT',
 'FLAG_UPDOWN',
 'SIGN',
 'LOG',
 'EXP',
 'DIFF',
 'LAG',
 'MIN',
 'MAX',
 'SUM',
 'ROC',
 'MAD',
 'STDEV',
 'SMA',
 'EMA',
 'RMA',
 'WMA',
 'HMA',
 'DEMA',
 'TEMA',
 'MA',
 'RSI',
 'ADX',
 'PLUSDI',
 'MINUSDI',
 'TRANGE',
 'ATR',
 'NATR',
 'SAR',
 'CCI',
 'CMF',
 'MFI',
 'BOP',
 'BBANDS',
 'KELTNER',
 'KER',
 'KAMA',
 'MACD',
 'PPO',
 'SLOPE',
 'RVALUE',
 'FORECAST',
 'STOCH',
 'STREAK',
 'EVAL']

In [16]:
# Sample code for indicator

from mintalib.core import wrap_indicator
from mintalib.model import FuncIndicator

@wrap_indicator(core.SMA)
def SMA(period: int = 3):
    return FuncIndicator.wrap(core.SMA, params=locals())


In [17]:
PREAMBLE = '''
"""
Mintalib Indicators

Indicators names are upper case.

Indicators offer a composable interface where a function is bound with its calculation parameters.
When instantiated with parameters an indicator yields a callable that can be applied to prices or series data.
Indicators support the `@` operator as syntactic sugar to apply the indicator to data.
So for example `SMA(50) @ prices` can be used to compute the 50 period simple moving average on `prices`,
insted of `SMA(50)(prices)`.

"""

# Do not edit! This file was generated by make-indicators.ipynb

from . import core
from .core import wrap_indicator
from .model import FuncIndicator

nan = float('nan')


'''


def make_indicator(func, name: str = None, preamble=False, verbose=False):
    if name is None:
        name = func.__name__.upper()

    fname = func.__qualname__
    module = func.__module__.rpartition(".")[2]
    qname = f"{module}.{fname}"

    signature = Signature.from_callable(func)
    parameters = list(signature.parameters.values())

    if verbose:
        print("signature", signature)
        print("parameters", parameters)

    newparams = []
    ftype = parameters[0].name

    def annotate(param):
        if param.annotation == "int":
            return param.replace(annotation=int)

        if param.annotation == "str":
            return param.replace(annotation=str)

        if type(param.default) in (int, float, bool):
            return param.replace(annotation=type(param.default))

        return param


    newparams = parameters[1:]
    newparams = [annotate(p) for p in newparams]

    if verbose:
        print("newparams", newparams)

    newsig = Signature(newparams)

    def argument(arg):
        return ftype if arg == "self" else f"{arg}=self.{arg}"

    code = PREAMBLE if preamble else ""
    # code = "# noinspection PyPep8Naming\n"

    code += f"@wrap_indicator({qname})\n"
    code += f"def {name}{newsig}:\n"
    code += f"    return FuncIndicator({qname}, params=locals())\n"


    return code


TEXT(make_indicator(core.EMA))

@wrap_indicator(core.EMA)
def EMA(period: int, *, adjust: bool = False, item: str = None):
    return FuncIndicator(core.EMA, params=locals())

In [18]:
def test_indicator(func, name=None, verbose=False):
    code = make_indicator(func, name=name, preamble=True, verbose=verbose)
    return new_module("mintalib.functions", code)


test_indicator(core.EMA)

<module 'mintalib.functions'>

In [19]:

def make_indicators(names=None):
    if names is None:
        names = core_functions()

    output = PREAMBLE

    buffer = pformat(names, width=75, compact=True, indent=4)
    buffer = buffer.replace("[", " ").replace("]", "")
    output += f"__all__ = [\n{buffer}\n]\n"

    output += "\n\n"

    for name in names:
        func = getattr(core, name)
        text = make_indicator(func)
        output += text + "\n\n"

    # output += "__all__ = [name for name in dir() if name.isupper()]\n"

    return output


code = make_indicators()

TEXT(code[:1024] +  " ...")


"""
Mintalib Indicators

Indicators names are upper case.

Indicators offer a composable interface where a function is bound with its calculation parameters.
When instantiated with parameters an indicator yields a callable that can be applied to prices or series data.
Indicators support the `@` operator as syntactic sugar to apply the indicator to data.
So for example `SMA(50) @ prices` can be used to compute the 50 period simple moving average on `prices`,
insted of `SMA(50)(prices)`.

"""

# Do not edit! This file was generated by make-indicators.ipynb

from . import core
from .core import wrap_indicator
from .model import FuncIndicator

nan = float('nan')


__all__ = [
    'AVGPRICE', 'TYPPRICE', 'WCLPRICE', 'MIDPRICE', 'PRICE', 'CROSSOVER',
    'CROSSUNDER', 'FLAG_ABOVE', 'FLAG_BELOW', 'FLAG_INVERT', 'FLAG_UPDOWN',
    'SIGN', 'LOG', 'EXP', 'DIFF', 'LAG', 'MIN', 'MAX', 'SUM', 'ROC', 'MAD',
    'STDEV', 'SMA', 'EMA', 'RMA', 'WMA', 'HMA', 'DEMA', 'TEMA', 'MA',
    'RSI', 'ADX', 'PLU

In [20]:
code = make_indicators()

indicators = new_module("mintalib.indicators", code)

dir(indicators)

['ADX',
 'ATR',
 'AVGPRICE',
 'BBANDS',
 'BOP',
 'CCI',
 'CMF',
 'CROSSOVER',
 'CROSSUNDER',
 'DEMA',
 'DIFF',
 'EMA',
 'EVAL',
 'EXP',
 'FLAG_ABOVE',
 'FLAG_BELOW',
 'FLAG_INVERT',
 'FLAG_UPDOWN',
 'FORECAST',
 'FuncIndicator',
 'HMA',
 'KAMA',
 'KELTNER',
 'KER',
 'LAG',
 'LOG',
 'MA',
 'MACD',
 'MAD',
 'MAX',
 'MFI',
 'MIDPRICE',
 'MIN',
 'MINUSDI',
 'NATR',
 'PLUSDI',
 'PPO',
 'PRICE',
 'RMA',
 'ROC',
 'RSI',
 'RVALUE',
 'SAR',
 'SIGN',
 'SLOPE',
 'SMA',
 'STDEV',
 'STOCH',
 'STREAK',
 'SUM',
 'TEMA',
 'TRANGE',
 'TYPPRICE',
 'WCLPRICE',
 'WMA',
 '__all__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'core',
 'nan',
 'wrap_indicator']

In [21]:
assert callable(indicators.SMA(20))

res = indicators.SMA(20) @ prices

assert len(res) > 0


In [22]:
outfile = PKGDIR / "indicators.py"

print(f"Updating {outfile.name} ...")

outfile.write_text(code)

Updating indicators.py ...


8095

In [23]:
from mintalib import indicators

assert callable(indicators.SMA(20))

res = indicators.SMA(20) @ prices

assert len(res) > 0